In [32]:
import requests

def chat_with_private_model(product_name: str, tone_style: str = "活泼甜美"):
    url = 'http://localhost:3000/api/chat/completions'
    headers = {
        'Authorization': f'Bearer sk-830e3768a6334044bcbc8b32f9888be6',
        'Content-Type': 'application/json'
    }
    data = {
      "model": "deepseek-r1:1.5b",
      "messages": [
        {
            "role": "user","content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"
        }
      ]
    }
    response = requests.post(url, headers=headers, json=data)

    #return response.json()

    response_message_content = response.json()["choices"][0]["message"]["content"]

    if(response_message_content):
        print(f"[模型生成结果] {response_message_content}")
        # --- START: 添加 JSON 提取和解析逻辑 ---
        json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message_content, re.DOTALL)

        if json_string_match:
            extracted_json_content = json_string_match.group(1)
            try:
                final_response = json.loads(extracted_json_content)
                print("Agent: 任务完成，成功解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError as e:
                print(f"Agent: 提取到JSON块但解析失败: {e}")
                print(f"尝试解析的字符串:\n{extracted_json_content}")
        else:
            # 如果没有匹配到 ```json 块，尝试直接解析整个 content
            try:
                final_response = json.loads(response_message_content)
                print("Agent: 任务完成，直接解析最终JSON文案。")
                return json.dumps(final_response, ensure_ascii=False, indent=2)
            except json.JSONDecodeError:
                print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
        # --- END: 添加 JSON 提取和解析逻辑 ---
    else:
        print("Agent: 未知响应，可能需要更多交互。")

    return "未能成功生成文案。"

In [37]:
import re
import json

def extract_json_and_format(input_string):

    try:
        # 解析JSON字符串
        json_data = json.loads(input_string)

        # 构建Markdown格式结果
        result = f"# {json_data['标题']}\n\n"
        result += f"{json_data['正文']}\n\n"
        result += f"**标签**: {', '.join(json_data['相关标签'])}\n\n"
        result += f"**表情符号**: {' '.join(json_data['相关表情符号'])}"

        return result

    except json.JSONDecodeError:
        return "JSON解析失败"
    except KeyError:
        return "JSON结构不完整"

In [33]:
#test query local docker ollama
product_name_3 = "视黄醇抗皱晚霜"
tone_style_3 = "甜美可爱"
result_3 = chat_with_private_model(product_name_3, tone_style_3)

print(result_3)

[模型生成结果] <think>
好，用户让我写一篇关于“视黄醇抗皱晚霜”的小红书文案，要求语气甜美可爱，包含标题、正文，至少5个相关标签和5个表情符号。而且要以完整的JSON格式输出，并用markdown代码块包裹。

首先，我得理解用户的需求。他们可能是一个喜欢时尚产品的小白，想要在小红书上快速吸引关注，分享产品的使用体验和效果。甜美的语气能增加互动率，所以我要让文案充满活力和亲切感。

接下来，标题部分要突出抗皱的效果和细腻的感觉，同时用可爱的情绪词开头，比如“超嫩细腻”这样的词汇，吸引眼球。正文需要详细说明产品成分，强调高浓度的提亮效果，可能加入一些具体的例子，比如在晚上使用后皮肤变得光滑细腻。还要提到适合不同肤质的人群，强调温和非刺激性的特点，这样读者觉得适用性广。

然后，相关标签要选择自然、抗皱、细腻、紧致这些词汇，同时用可爱的表情符号来增加视觉吸引力。另外，可以加入一些关于坚持提升效果的鼓励语句，让文案更有动力感。

最后，我需要确保整个结构清晰，有标题和正文，至少五个标签和表情符号，整个内容都要用简洁有力的语言表达出来，并且符合小红书的风格，可能还需要添加一些相关的话题标签来扩大搜索范围。

总结一下，步骤应该是先构思好文案内容，再按照要求调整格式和语言，确保每个要素都满足用户的需求。
</think>

```json{
    "标题": "超嫩细腻的抗皱效果，用视黄醇晚霜让你的皮肤变得细腻有神！",
    "正文": "姐妹们！看这✨ 视黄醇抗皱晚霜，真的是你的皮肤最温柔的选择！它能深层提亮肌肤，让肤色瞬间变细腻光滑！晚上用起来还能有效紧致皮肤。无论是敏感肌还是油性肌，这款产品都能让你的皮肤焕发光彩！关键是它是温和超非刺激性的，让你轻松享受抗皱的效果。用完后皮肤真的比以前更紧实， super嫩 super smooth！✨ 嘿Oh~ 让您的锁骨线消失，让你的气质瞬间提升！✨ 无论是日常护肤还是日常护理，这是一款非常不错的选择哦！#自然 beauty #抗皱 #细腻感 #紧致肌肤",
    "相关标签": ["自然", "抗皱", "细腻", "紧致", "皮肤"],
    "相关表情符号": ["萌", "甜蜜", "温柔", "温暖", "神奇"]
}
```
Agent: 任务完成，成功解析最终JSON文案。
{
  "标题": "

In [38]:
markdown_note = extract_json_and_format(result_3)

print("--- <UNK> (Markdown) ---")
print(markdown_note)

--- <UNK> (Markdown) ---
# 超嫩细腻的抗皱效果，用视黄醇晚霜让你的皮肤变得细腻有神！

姐妹们！看这✨ 视黄醇抗皱晚霜，真的是你的皮肤最温柔的选择！它能深层提亮肌肤，让肤色瞬间变细腻光滑！晚上用起来还能有效紧致皮肤。无论是敏感肌还是油性肌，这款产品都能让你的皮肤焕发光彩！关键是它是温和超非刺激性的，让你轻松享受抗皱的效果。用完后皮肤真的比以前更紧实， super嫩 super smooth！✨ 嘿Oh~ 让您的锁骨线消失，让你的气质瞬间提升！✨ 无论是日常护肤还是日常护理，这是一款非常不错的选择哦！#自然 beauty #抗皱 #细腻感 #紧致肌肤

**标签**: 自然, 抗皱, 细腻, 紧致, 皮肤

**表情符号**: 萌 甜蜜 温柔 温暖 神奇
